# 知识工程-问答系统设计与实现大作业
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本文档理论上支持多个格式，包括ipynb, docx, pdf 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对原本代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 数据准备

非常好的助教已经帮我们down好了数据，注意到有四个文件
- passages_multi_sentences.json (文档库) 
- train.json (问答语料) 
- train_questions.txt (问题分类训练语料) 
- test_questions.txt (问题分类测试语料) 

我们自己查看一下数据格式

![alt text](image.png)

![alt text](image-1.png)


| 文件名 (File Name)             | 来源数据集 (Source Dataset)                                  | 数据集描述 (Dataset Description)                                                                                                                               | 文件格式 (File Format)                                                                                                                                  |
| :----------------------------- | :----------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `passages_multi_sentences.json` | 百度 DuReader V2.0 (筛选处理后)                     | 包含用于检索和答案抽取的文档库，这些文档被分割成了多个句子 。                                                                                             | 实际上是JSONL，每一行是JSON 格式。每一个json有pid字段和document 字段，document是句子字符串的列表，每个字符串逻辑上是一个Sentence。                                                                                                                       |
| `train.json`                   | 百度 DuReader V2.0 (筛选处理后)                     | 问答语料，包含问题、对应的答案以及相关的文档 `pid` 。用于训练检索评估、候选答案句排序和答案抽取模块。                                                    | 也是JSONL，每一行的对象是question、pid、          answer_sentence、   answer、qid组成的，其中answer_sentence是句子的列表，但是一般是一个句子，answer和question是单个字符串，pid和qid是int                                                          |
| `train_questions.txt`          | 哈工大信息检索研究室问答系统问题集                      | 用于训练问题分类器的问题及对应的类别标签 。类别标签形如 `HUM_PERSON`，下划线前为粗分类标签 。                                                   | 文本文件 (.txt)。每行可能是一个标签和一个问题，由制表符或空格分隔。标签看起来是这个问题的类型，比如DES_DEFINITION是要求别人描述定义的问题。                                                                                                       |
| `test_questions.txt`           | 哈工大信息检索研究室问答系统问题集                      | 用于测试问题分类器性能的问题及对应的类别标签 。                                                                    | 与`train_questions.txt`一样。                                                                                                     |


主要来自两个数据集 ：

- DuReader V2.0：这是百度在2017年发布的一个阅读理解数据集 。本次实验使用的 passages_multi_sentences.json 和 train.json 就是从这个数据集中筛选和处理得到的 。
- 哈工大信息检索研究室问答系统问题集：这是一个由哈工大信息检索研究室在2004年公开的数据集 。实验中的问题分类语料（即 train_questions.txt 和 test_questions.txt）来源于此 。

## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image-2.png)


很好，这次代码格式化没有报错。

Pylance 似乎也没有明显问题。

## 实验环境准备

采用上次的作业专属环境，为了跑通最新方法，使用3.12 和 torch 2.7

```bash
conda create -n assignments python=3.12
conda activate assignments
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

本次作业似乎没有新的依赖，只是用到了 torch


## 原理回顾和课件复习



在作业9. 中我们进行了简单的复习，这里不再赘述。

## 1. 建立文档检索系统

首先我们来理解这个子任务的要求，
我注意到，train.json中的样本的pid，其实就是passages中的pid，是一一对应的。
我们要构建一个函数，输入train.json中的question，中间经过关键词提取, 系统读取passages_multi_sentences.json中的文档库，返回与问题相关的文档的pid列表， 使用Recall指标来评估检索系统的性能。

这里没有让我们用深度学习，而是传统的检索。

首先需要去除停用词，停用词是指在文本中出现频率非常高，但对文本意义贡献不大的词语（如“的”、“是”、“在”等）。去除它们可以减少噪音，降低计算量。注意到助教已经提供了停用词列表 `stopwords.txt`，我们可以直接使用。

在助教的作业要求中，建议我们使用Whoosh